1. TreeOptimizerクラスを改変
    1. 1つの木は深さ1，構成要素は数学演算子とその引数に用いる定数か引数
    2. リスト形式で，複数の木を保持するように変更
    3. 各々の木の出力を計算する際に，他の木の出力を引数に持っているときのために，variable_listにあらかじめ用意しておく

2. 変更したTreeOptimizerクラスで作成したSGDの学習は正しくできた．
3. Momentum の学習で各木の出力がNoneになってしまい，正しく学習ができない --> 更新式が間違っていた
4. 突然変異の設計
    1. 木の各要素の変異
        1. 関数は関数にしか変化しない
        2. 変数は変数，定数に変化する
        3. 定数は変数，定数に変化する
    2. ランダムな行の削除
    3. ランダムな行に追加

In [1]:
from local.node import Node
from local.constnode import ConstNode
from local.varnode import VarNode
from local.funcnode import FuncNode
from local.func import *

import os
import glob
import copy
import random
import pickle
import graphviz
import datetime
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, losses, metrics, optimizers

In [2]:
class TreeOptimizer:
    # 一つの木には1つの数学演算子とその引数(定数,変数)で構成する
    mutate_probability = 0.1
    max_depth = 1

    def __init__(self):
        # 構成する木の本数を決定
        self.tree_num = random.randint(1,10)
        # 仮枠の木をはやす
        self.tree_list = [FuncNode(max_depth=self.max_depth) for i in range(self.tree_num)]
        # 引数を決める際に，他の木の出力を用いられるように，set_random_idの時に用いる辞書を作成
        self.tree_dict = {i : {"node_type" : "function"} for i in range(self.tree_num)}

        self.set_node_id()
        self.set_random_id()
        self.set_save_flag()
        self.initialize_state()

    def build(self, var_list):
        for i in range(self.tree_num):
            self.tree_list[i].save_flag = True
            self.tree_list[i].build(var_list)
            self.iteration_slots = list()
            for V in var_list:
                self.iteration_slots.append( tf.Variable( tf.zeros(V.shape) ) )
            self.tree_list[i].is_built = True
    
    def set_node_id(self):
        node_ids = [list() for i in range(self.tree_num)]
        for i in range(self.tree_num):
            self.tree_list[i].set_node_id(node_ids[i])

    def set_random_id(self):
        for i in range(self.tree_num):
            self.tree_list[i].set_random_id(self.tree_dict)

    # ノードの計算を保存するか決める関数
    def set_save_flag(self):
        # とりあえず，全部保存する方向でTrue
        for i in range(self.tree_num):
            # self.tree_list[i].set_save_flag(self.tree_dict)
            self.tree_list[i].save_flag = True

    def initialize_state(self):
        for i in range(self.tree_num):
            self.tree_list[i].is_built = False
            self.tree_list[i].iteration = 0.0

    def make_struct_dict(self):
        struct_dict = {i : dict() for i in range(self.tree_num)}
        for i in range(self.tree_num):
            self.tree_list[i].make_struct_dict(struct_dict[i])
        return struct_dict

    def make_variable_dict(self, slot_index: int):
        variable_dict = dict()
        for i in range(self.tree_num):
            variable_dict[i] = self.tree_list[i].slots[slot_index]
        return variable_dict
    
    def update_step(self, gradient, parameter, slot_index):
        # 複数の木で構成し，引数に他の木の出力を用いるときのために，
        # 作成するvariable_dictの要素に各木の出力を保持するように変更
        variable_dict = self.make_variable_dict(slot_index)
        variable_dict["gradient"] = gradient
        variable_dict["parameter"] = parameter
        self.iteration_slots[slot_index].assign_add(tf.ones(parameter.shape))
        variable_dict["iteration"] = self.iteration_slots[slot_index]

        for i in range(self.tree_num):
            if not self.tree_list[i].skip_flag:
                self.tree_list[i](variable_dict, slot_index)
            
        parameter.assign_sub(self.tree_list[self.tree_num-1].slots[slot_index])
    
    def apply_gradients(self, grad_and_vars):
        for i in range(self.tree_num):
            if self.tree_list[i].is_built == False:
                var_list = [GaV[1] for GaV in grad_and_vars]
                self.build(var_list)
        for i, (G, V) in enumerate(grad_and_vars):
            self.update_step(G, V, i)
            
    def mutation(self):
        # 突然変異を行う次元の選択
        mutate_index = random.choice(range(self.tree_num))

        p = random.random()
        # 遺伝子の削除 --> 削除して参照先がなくなってしまったときのために0として扱う 
        if p < 0.3:
            tmp = FuncNode(max_depth = 1, function = SignFunc())
            tmp.args[0] = ConstNode(depth = 1, constant = 0.0)
            tmp.skip_flag = True
            self.tree_list[mutate_index] = tmp
        # 遺伝子の追加
        elif p < 0.6:
            tmp = FuncNode(max_depth = 1)
            tmp.skip_flag = False
            self.tree_list.insert(mutate_index, tmp)
        # 要素の変更
        else:
            p = random.random()
            # 葉の要素の変更
            if p < 0.3:
                for i in range(len(self.tree_list[mutate_index].args)):
                    if random.choice([0, 1]):
                        self.tree_list[mutate_index].args[i] = VarNode(depth = 1)
                    else:
                        self.tree_list[mutate_index].args[i] = ConstNode(depth = 1)
            # 関数のみ変更
            elif p < 0.6:
                args = copy.deepcopy(self.tree_list[mutate_index].args)
                self.tree_list[mutate_index] = FuncNode(max_depth = 1)
                self.tree_list[mutate_index].skip_flag = False
                if len(args) >= len(self.tree_list[mutate_index].args):
                    for i in range(len(self.tree_list[mutate_index].args)):
                        self.tree_list[mutate_index].args[i] = args[i]
                else:
                    self.tree_list[mutate_index].args[0] = args[0]
                    if random.choice([0, 1]):
                        self.tree_list[mutate_index].args[1] = VarNode(depth = 1)
                    else:
                        self.tree_list[mutate_index].args[1] = ConstNode(depth = 1)
            # 全て変更
            else:
                self.tree_list[mutate_index] = FuncNode(max_depth = 1)
                self.tree_list[mutate_index].skip_flag = False
                
        
        self.tree_num = len(self.tree_list)
        self.tree_dict = {i : {"node_type" : "function"} for i in range(self.tree_num)}

        for i in range(self.tree_num):
            for j in range(len(self.tree_list[i].args)):
                if isinstance(self.tree_list[i].args[j], VarNode) and self.tree_list[i].args[j].variable_id == None:
                    self.tree_list[i].set_random_id(self.tree_dict)
        
        self.set_node_id()
        self.initialize_state()


In [ ]:
tmp = TreeOptimizer()
tmp.tree_num, tmp.tree_list